# Monads laws in Raku

Anton Antonov   
MathematicaForPrediction at WordPress  
RakuForPrediction at WordPress   
November 2025

----

## Introduction

I participated this in the [Wolfram Technology Conference 2025](http://www.wolfram.com/events/technology-conference/2025/). 
My talk was titled "Applications of Monadic Programming", a shorter version of a similarly named presentation ["Applications of Monadic Programming, Part 1, Questions & Answers"](https://www.youtube.com/watch?v=Xz5B4B0kVco), [AAv5], which I recorded and posted three months ago.  

After the conference I decided that it is a good idea to rewrite and re-record the presentation with a Raku-centric exposition.
(I have done that before, see: ["Simplified Machine Learning Workflows Overview (Raku-centric)"](https://www.youtube.com/watch?v=p3iwPsc6e74), [AAv4].)

That effort requires the so called [***Monad laws***](https://wiki.haskell.org/index.php?title=Monad_laws) to be verified that they apply to certain constructs of the Raku language. This document (notebook) defines the Monad laws and provides several verifications for different combinations of operators and coding styles.

This document focuses on built-in Raku features that can be used in monadic programming. We will not cover Raku packages that enhance Raku's functionality or syntax for monadic programming.

### Context

Before going further, let us list the applications of monadic programming we consider:

1. Graceful failure handling

2. Rapid specification of computational workflows

3. Algebraic structure of written code

**Remark:** Those applications are discussed in [AAv5] and its future Raku version: 

As Data Science (DS) or Machine Learning (ML) tools maker, I am interested of three applications, but as "simple data scientist" I am mostly interested in 2.
That said, a large part of my Raku programming has been dedicated to rapid and reliable code generation for DS and ML by leveraging the algebraic structure of corresponding software monads. (This is discussed in [AAv2, AAv3, AAv4].)

### Dictionary

- **Monadic programming**   
  A method for organizing computations as a series of steps, where each step generates a value along with additional information about the computation, such as possible failures, non-determinism, or side effects. See [Wk1].

- **Monadic pipeline**   
  Chaining of operations with a certain syntax.

- **Uniform Function Call Syntax (UFCS)**  
  A feature that allows both free functions and member functions to be called using the same `object.function()` method call syntax. 

- **Method-like call**   
  Same as UFCS. A Raku example: `[3, 4, 5].&f1.$f2`.

### Verifications overview

Raku -- as expected -- has multiple built-in mechanisms for doing monadic programming. A few of those mechanisms are "immediate", other require a certain coding style or very direct and simple definitions. Not all the Monad law verifications are have to be known (or understood) by a programmer. Here is a table that summarizes them:

<table border="1">
  <tr>
    <th>Type</th>
    <th>Description</th>
  </tr>
  <tr>
    <td><code>Array</code> and <code>==&gt;</code></td>
    <td>Most immediate, clear-cut</td>
  </tr>
  <tr>
    <td><code>&amp;unit</code> and <code>&amp;bind</code></td>
    <td>Definitions according to Monad laws</td>
  </tr>
  <tr>
    <td><code>Any</code> and <code>andthen</code></td>
    <td>General, built-in monad!</td>
  </tr>
  <tr>
    <td>Styled OOP</td>
    <td>Standard and straightforward</td>
  </tr>
</table>



---

## What is a monad? (informally speaking)

Many programmers are familiar with monadic pipelines, although, they might know them under different names. This section has monadic pipeline examples from Unix and R, that should help the more formal definitions in the next section.

### Unix examples

Most (old and/or Raku) programmers are familiar with Unix programming. Hence, they are familiar with monadic pipelines.

#### Unix pipeline

The [Unix pipeline](https://en.wikipedia.org/wiki/Pipeline_(Unix)) semantics and syntax was invented and introduced soon after the first Unix release.
Monadic pipelines (or uniform function call) have very similar motivation and syntax.

Here is an example of Unix pipeline in which the output of one shell program is the input for the next:

```
find . -name "*nb" | grep -i monad | xargs -Iaaa date -r aaa
```

In [178]:
#% bash
find . -name "*nb" | grep -i monad | xargs -Iaaa date -r aaa

Sun Nov  9 13:48:37 EST 2025
Fri Nov  7 15:30:55 EST 2025


The command:

1. Finds in the current directory all files with names that finish with "nb"
2. Picks from the list produces by 1 only the rows that contain the string "monad"
3. Gives the dates of modification of those files

#### Reverse-Polish calculator

One of the oldest surviving Unix language program is [`dc` (desktop calculator)](https://en.wikipedia.org/wiki/Dc_(computer_program))
that uses reverse-Polish notation. Here is an example of command given to `dc` that prints out `32`, i.e. `(3 + 5) * 4`: 

```
3 5 + 4 * p
```

We can see that `dc` command as a pipeline: 

- The numbers are functions that place in the context (which is a stack) the corresponding values 

- The space between the symbols is the pipeline constructor


### Data wrangling

[Posit](https://posit.co)'s constellation of R packages ["tidyverse"](https://tidyverse.org) facilitates
pipeline construction of data wrangling workflows. Here is an example of R-tidyverse code that is generated from natural language commands -- each row corresponds to pipeline segment:

In [220]:
use DSL::Translators;

my $commands = q:to/END/;
use dataset dfTitanic;
rename columns passengerAge as age, passengerSex as sex, passengerClass as class;
filter by age > 10;
counts;
END

$commands ==> ToDSLCode(format=>'code')

dfTitanic %>%
dplyr::rename(age = passengerAge, sex = passengerSex, class = passengerClass) %>%
dplyr::filter(age > 10) %>%
dplyr::count()

Here is the Raku -- non-monadic -- version of the code above:

In [223]:
$commands ==> ToDSLCode(format=>'code', default-targets-spec => 'Raku')

$obj = dfTitanic ;
$obj = rename-columns( $obj, %("passengerAge" => "age", "passengerSex" => "sex", "passengerClass" => "class") ) ;
$obj = $obj.grep({ $_{"age"} > 10 }).Array ;
$obj = $obj.elems

---

## What is a monad? (formally speaking)


#### The monad definition

In this document a monad is any set of a symbol $m$ and two operators *unit* and *bind* that adhere to the monad laws. (See the next sub-section.) The definition is taken from [Wk1] and [PW1] and phrased in Raku terms. In order to be brief, we deliberately do not consider the equivalent monad definition based on *unit*, *join*, and *map* (also given in [PW1].) 

Here are operators for a monad associated with a certain class `M`:

1. monad *unit* function ("return" in Haskell notation) is `&unit(x) = M.new(x)`

2. monad *bind* function (">>=" in Haskell notation) is a rule like `&bind(M:D $x, &f) = &f(x)` with `&f($x) ~~ M:D` giving `True`.

Note that:

  - the function `&bind` unwraps the content of `M` and gives it to the function `&f`;

  - the functions given as second argument to `&bind` (see`&f`) are responsible to return as results instances of the monad class `M`.

Here is an illustration formula showing a ***monad pipeline***:

$$
M(x) \xRightarrow[\text{Bind}(M(x), f_1)]{\text{}} f_1 
\xRightarrow[\text{Bind}(M(x), f_2)]{\text{}} f_2
\xRightarrow[\text{Bind}(M(x), f_3)]{\text{}} f_3
\xRightarrow[\text{Bind}(M(x), f_3)]{\text{}} \cdots
\xRightarrow[\text{Bind}(M(x), f_k)]{\text{}} f_k
$$

From the definition and formula it should be clear that if for the result `f(x)` of `&bind` the test `f(x) ~~ M:D` is `True` then the result is ready to be fed to the next binding operation in monad's pipeline. Also, it is easy to program the pipeline functionality with `reduce`:

```raku
reduce(&bind, M.new(3), [&f1, &f2, $f3])
```

Instead of `reduce`, "method-like call" can be used:

In [131]:
[6, 3, 12].&{ $_.elems }.&{ sqrt($_) }.&{ $_ ** 3 }

5.196152422706631

**Remark** A simpler version of the code above is: `[6, 3, 12].elems.sqrt.&{ $_ ** 3 }`.

#### The monad laws

The monad laws definitions are taken from [[H1](https://wiki.haskell.org/Monad_laws)] and [H3].[ ](https://wiki.haskell.org/Monad_laws)In the monad laws given below "⟹" is for monad's binding operation and $(x|\rightarrow \text{expr})$ is for a function in anonymous form.

Here is a table with the laws:

<!-- 
[
 { name => 'Left identity',  :input( 'unit m ⟹ f'   ), :output( 'f m'  )},
 { name => 'Right identity', :input( 'm ⟹ unit'     ), :output( 'm'    )},
 { name => 'Associativity',  :input( '(m ⟹ f) ⟹ g' ), :output( 'm ⟹ (x ⟼ f x ⟹ g)' )},
] ==> to-html(field-names => <name input output>)
-->

<table border="1"><thead><tr><th>name</th><th>LHS</th><th>RHS</th></tr></thead><tbody><tr><td>Left identity</td><td>unit m ⟹ f</td><td>f m</td></tr><tr><td>Right identity</td><td>m ⟹ unit</td><td>m</td></tr><tr><td>Associativity</td><td>(m ⟹ f) ⟹ g</td><td>m ⟹ (x ⟼ f x ⟹ g)</td></tr></tbody></table>

----

## `Array` and `==>` 

The monad laws are satisfied in Raku for:
- Every function `f` that takes array argument and returns an array
- The unit operation being `Array`
- The forward feed operator (`==>`) being the binding operation

<table border="1">
  <tr>
    <th>Name</th>
    <th>Input</th>
    <th>Output</th>
  </tr>
  <tr>
    <td>Left identity</td>
    <td><code>Array($a) ==> &amp;f()</code></td>
    <td><code>&amp;f($a)</code></td>
  </tr>
  <tr>
    <td>Right identity</td>
    <td><code>$a ==> { Array($_) }()</code></td>
    <td><code>$a</code></td>
  </tr>
  <tr>
    <td>Associativity LHS</td>
    <td><code>Array($a) ==> &amp;f1() ==> &amp;f2()</code></td>
    <td><code>&amp;f2(&amp;f1($a))</code></td>
  </tr>
  <tr>
    <td>Associativity RHS</td>
    <td><code>Array($a) ==> { &amp;f($_) ==> &amp;f2() }()</code></td>
    <td><code>&amp;f2(&amp;f1($a))</code></td>
  </tr>
</table>

Here is an example:

In [ ]:
#% html

# Operators in the monad space
my &f =    { Array($_) >>~>> '_0' }
my &f1 =   { Array($_) >>~>> '_1' }
my &f2 =   { Array($_) >>~>> '_2' }

# Some object
my $a = 5; #(3, 4, 'p');

# Verification table
my @tbl =
 { name => 'Left identity',     :input( Array($a) ==> &f()                    ), :output( &f($a)       )},
 { name => 'Right identity',    :input( $a ==> { Array($_) }()                ), :output( $a           )},
 { name => 'Associativity LHS', :input( Array($a) ==> &f1() ==> &f2()         ), :output( &f2(&f1($a)) )},
 { name => 'Associativity RHS', :input( Array($a) ==> { &f1($_) ==> &f2() }() ), :output( &f2(&f1($a)) )}
;

@tbl ==> to-html(field-names => <name input output>)

name,input,output
Left identity,5_0,5_0
Right identity,5,5
Associativity LHS,5_1_2,5_1_2
Associativity RHS,5_1_2,5_1_2


----

## `&unit` and `&bind`

Alternatively, we can use the definitions of `&unit` and `&bind` below:

In [ ]:
#% html

# Monad operators
my &unit = { Array($_) };
my &bind = { $^b($^a) };

# Operators in the monad space
my &f  = { Array($_) >>~>> '_0' }
my &f1 = { Array($_) >>~>> '_1' }
my &f2 = { Array($_) >>~>> '_2' }

# Some object
my $a = (3, 4, 'p');

# Verification table
my @tbl =
 { name => 'Left identity',     :input( &bind( &unit($a), &f)                      ), :output( &f($a)       )},
 { name => 'Right identity',    :input( &bind( $a, &unit)                          ), :output( $a           )},
 { name => 'Associativity LHS', :input( &bind( &bind( &unit($a), &f1), &f2)        ), :output( &f2(&f1($a)) )},
 { name => 'Associativity RHS', :input( &bind( &unit($a), { &bind(&f1($_), &f2) }) ), :output( &f2(&f1($a)) )}
;

@tbl ==> to-html(field-names => <name input output>)

name,input,output
Left identity,3_04_0p_0,3_04_0p_0
Right identity,34p,34p
Associativity LHS,3_1_24_1_2p_1_2,3_1_24_1_2p_1_2
Associativity RHS,3_1_24_1_2p_1_2,3_1_24_1_2p_1_2


----

## `Any` and `andthen` 

The operator `andthen` is similar to the feed operator `==>`. For example:

```raku
my $hw = "  hello world  ";
$hw andthen .trim andthen .uc andthen .substr(0,5) andthen .say
```

From the [documentation](https://docs.raku.org/language/operators#infix_andthen):

> The `andthen` operator returns `Empty` if the first argument is undefined, otherwise the last argument. The last argument is returned as-is, without being checked for definedness at all. Short-circuits. The result of the left side is bound to `$_` for the right side, or passed as arguments if the right side is a `Callable`, whose `count` must be 0 or 1. 

Note that these two expressions are equivalent:

```raku
$a andthen .&f1 andthen .&f2;
$a andthen &f1($_) andthen &f2($_);
```

A main feature `andthen` is to return `Empty` if its first argument is not defined.
That is, actually, very "monadic" -- *graceful handling of errors* is one of the main reasons of use Monadic programming. 
It is also limiting, because the monad failure is "just" `Empty`. 
(That is mostly a theoretical limitation; in practice Raku has many other elements, like, `notandthen`, that can shape the workflows to programmer's desires.)

The Monad laws hold for `Any.new` as the unit operation and `andthen` as the binding operation.

In [ ]:
#% html
# Operators in the monad space
my &f  = { Array($_) >>~>> '_0' }
my &f1 = { Array($_) >>~>> '_1' }
my &f2 = { Array($_) >>~>> '_2' }

# Some object
my $a = (3, 4, 'p');

# Verification table
my @tbl =
{ name => 'Left identity',     :input( '$a andthen .&f'                   ), :output( {$a andthen .&f}().raku                    )},
{ name => 'Right identity',    :input( '$a andthen {$_}'                  ), :output( {$a andthen {$_}}().raku                   )},
{ name => 'Associativity LHS', :input( '$a andthen .&f1 andthen .&f2)'    ), :output( {$a andthen .&f1 andthen .&f2}().raku      )},
{ name => 'Associativity RHS', :input( '$a andthen { .&f1 andthen .&f2 }' ), :output( {$a andthen { .&f1 andthen .&f2  }}().raku )}
;

@tbl ==> to-html(field-names => <name input output>)

name,input,output
Left identity,$a andthen .&f,"[""3_0"", ""4_0"", ""p_0""]"
Right identity,$a andthen {$_},"$(3, 4, ""p"")"
Associativity LHS,$a andthen .&f1 andthen .&f2),"[""3_1_2"", ""4_1_2"", ""p_1_2""]"
Associativity RHS,$a andthen { .&f1 andthen .&f2 },"[""3_1_2"", ""4_1_2"", ""p_1_2""]"


----

## Monad class and method call

Raku naturally supports method chaining using dot notation (`.`) for actual methods defined on a class or type.
Hence, a more "standard" way is to use a monad class, say `M`, and method call:

- `M.new(...)` plays the monad unit role -- i.e. it uplifts objects into monad's space

- `$m.f(...)` (where `$m ~~ M:D`) plays the binding role if all methods of `M` return `M:D` objects

The axioms verification needs to be done using a particular class definition format (see the example below):

**1.** Left identity applies: 

`M.new($x).f` does mean application of `M.f` to `$x`.

**2.** Right identity applies by using `M.new`
   
**3.** Associativity axiom holds

For RHS, again, method-like call (call as method) is used.


Here is an example:

In [287]:
#% html

# Monad class definition
my class M { 
    has $.context;
    multi method new($context) { self.bless(:$context) }
    multi method new(M:D $m) { self.bless(context => $m.context) }
    method f() { $!context = $!context >>~>> '_0'; self}
    method f1() { $!context = $!context >>~>> '_1'; self}
    method f2() { $!context = $!context >>~>> '_2'; self}
}

# Some object
my $a = 5; #[5, 3, 7];

# Verification table
my @tbl =
 { name => 'Left identity',     :input( 'M.new($a).f'              ), :output( M.new($a).f.raku                  )},
 { name => 'Right identity',    :input( 'my M:D $x .= new($a)'     ), :output( {my M:D $x .= new($a); $x}().raku )},
 { name => 'Associativity LHS', :input( '(M.new($a).f1).f2'        ), :output( (M.new($a).f1).f2.raku            )},
 { name => 'Associativity RHS', :input( 'M.new($a).&{ $_.f1.f2 }'  ), :output( M.new($a).&{ $_.f1.f2 }.raku      )}
;

@tbl ==> to-html(field-names => <name input output>)

name,input,output
Left identity,M.new($a).f,"M.new(context => ""5_0"")"
Right identity,my M:D $x .= new($a),M.new(context => 5)
Associativity LHS,(M.new($a).f1).f2,"M.new(context => ""5_1_2"")"
Associativity RHS,M.new($a).&{ $_.f1.f2 },"M.new(context => ""5_1_2"")"


----

## References

### Articles, blog posts

[Wk1] Wikipedia entry: [Monad (functional programming)](https://en.wikipedia.org/wiki/Monad_(functional_programming)), URL: [https://en.wikipedia.org/wiki/Monad_(functional_programming)](https://en.wikipedia.org/wiki/Monad_(functional_programming)) . 

[Wk2] Wikipedia entry: [Monad transformer](https://en.wikipedia.org/wiki/Monad_transformer), URL: [https://en.wikipedia.org/wiki/Monad_transformer](https://en.wikipedia.org/wiki/Monad_transformer) .

[H1] Haskell.org article: [Monad laws,](https://wiki.haskell.org/Monad_laws) URL: [https://wiki.haskell.org/Monad_laws](https://wiki.haskell.org/Monad_laws). 

[SH2] Sheng Liang, Paul Hudak, Mark Jones, ["Monad transformers and modular interpreters",](http://haskell.cs.yale.edu/wp-content/uploads/2011/02/POPL96-Modular-interpreters.pdf) (1995), Proceedings of the 22nd ACM SIGPLAN-SIGACT symposium on Principles of programming languages. New York, NY: ACM. pp. 333--343. doi:10.1145/199448.199528.

[PW1] Philip Wadler, ["The essence of functional programming"](https://page.mi.fu-berlin.de/scravy/realworldhaskell/materialien/the-essence-of-functional-programming.pdf), (1992), 19'th Annual Symposium on Principles of Programming Languages, Albuquerque, New Mexico, January 1992.

[RW1] Hadley Wickham et al., [dplyr: A Grammar of Data Manipulation](https://github.com/tidyverse/dplyr), (2014), [tidyverse at GitHub](https://github.com/tidyverse), URL: [https://github.com/tidyverse/dplyr](https://github.com/tidyverse/dplyr) .
       (See also, [http://dplyr.tidyverse.org](http://dplyr.tidyverse.org) .)

[AA1] Anton Antonov, ["Monad code generation and extension"](https://mathematicaforprediction.wordpress.com/2017/06/23/monad-code-generation-and-extension/), (2017), [MathematicaForPrediction at WordPress](https://mathematicaforprediction.wordpress.com).

### Packages

[AAp1] Anton Antonov, [MonadMakers](https://resources.wolframcloud.com/PacletRepository/resources/AntonAntonov/MonadMakers/), Wolfram Language paclet, (2023), [Wolfram Language Paclet Repository](https://resources.wolframcloud.com/PacletRepository/).

[AAp2] Anton Antonov, [StatStateMonadCodeGeneratoreNon](https://github.com/antononcube/R-packages/tree/master/StateMonadCodeGenerator) R package, (2019-2024), 
[GitHub/@antononcube](https://github.com/antononcube/).

### Videos

[AAv1] Anton Antonov, [Monadic Programming: With Application to Data Analysis, Machine Learning and Language Processing](https://www.youtube.com/watch?v=_cIFA5GHF58), (2017), Wolfram Technology Conference 2017 presentation. [YouTube/WolframResearch](https://www.youtube.com/@WolframResearch).

[AAv2] Anton Antonov, [Raku for Prediction](https://www.youtube.com/watch?v=frpCBjbQtnA), (2021), [The Raku Conference 2021](https://www.youtube.com/@therakuconference6823).

[AAv3] Anton Antonov, [Simplified Machine Learning Workflows Overview](https://www.youtube.com/watch?v=Xy7eV8wRLbE), (2022), Wolfram Technology Conference 2022 presentation. [YouTube/WolframResearch](https://www.youtube.com/@WolframResearch).

[AAv4] Anton Antonov, [Simplified Machine Learning Workflows Overview (Raku-centric)](https://www.youtube.com/watch?v=p3iwPsc6e74), (2022), Wolfram Technology Conference 2022 presentation. [YouTube/@AAA4prediction](https://www.youtube.com/@AAA4prediction).

[AAv5] Anton Antonov, [Applications of Monadic Programming, Part 1, Questions & Answers](https://www.youtube.com/watch?v=Xz5B4B0kVco), (2025), [YouTube/@AAA4prediction](https://www.youtube.com/@AAA4prediction).
